# Pipeline comparing two models RF and KNN and deploying the best one
Things to keep in mind for this to work:
- Need to define you personal project_id and region you are using (this is done at the beginning and end of the document)
- Change the name of model, temp, and data buckets to the one you are using (this is done at the beginning and end of the document)
- Need to upload `prediction_set.csv` and `trainning_set.csv` (named this exact way) to data bucket

### Installation
Install the packages required for executing this notebook.

In [2]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 595.6/595.6 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Check the versions of the packages you installed. The KFP SDK version should be >2.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.7.0
google-cloud-aiplatform==1.69.0
google_cloud_pipeline_components version: 2.17.0


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [24]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "your poject id" # make sure to change
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://your temp bucket"

#### Pipeline Component : Data Ingestion

In [5]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

#### Pipeline Component : Training-RF 

In [6]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_rf (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a Randome Forest with default parameters'''
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)        
    
    x_train, x_test, y_train, y_test = train_test_split(df.drop('Diagnosis', axis = 1), 
                                                    df['Diagnosis'], test_size=0.30, 
                                                    random_state=101)
    
    model_rf = RandomForestClassifier()
    model_rf.fit(x_train,y_train)
    
    y_pred=model_rf.predict(x_test)

    metrics_dict = {
        "accuracy": accuracy_score(y_test, y_pred)
    }
    logging.info(metrics_dict)  
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "RF"
   # Save the model
    m_file = out_model.path + ".pkl"
    with open(m_file, 'wb') as f:  
        pickle.dump(model_rf, f)   
    
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Training KNN

In [7]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_knn (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a KNN model with default parameters'''
    import pandas as pd
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    logging.info(df.columns)        
    
    x_train, x_test, y_train, y_test = train_test_split(df.drop('Diagnosis', axis = 1), 
                                                    df['Diagnosis'], test_size=0.30, 
                                                    random_state=101)
    model_knn = KNeighborsClassifier()
    model_knn.fit(x_train,y_train)
    
    y_pred = model_knn.predict(x_test)

    metrics_dict = {
        "accuracy": accuracy_score(y_test, y_pred)
    }
    
    logging.info(metrics_dict)  
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "KNN"
   # Save the model
    m_file = out_model.path + ".pkl"
    with open(m_file, 'wb') as f:  
        pickle.dump(model_knn, f)   
    
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Prediction-RF and KNN (not needed?)

In [8]:
""" @dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def predict_model(model: Input[Model], features: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle  
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
       
    filename = model.path + ".pkl"
        
    #Loading the saved model
    model_selected = pickle.load(open(filename, 'rb'))

    xNew = df[['Age', 'Gender', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Heart_Rate_bpm', 'Body_Temperature_C', 'Blood_Pressure_mmHg', 'Oxygen_Saturation_%']]

    dfcp = df.copy()   
    y_classes = model_selected.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig') """

#### Pipeline Component : Algorithm Selection 

In [19]:
@dsl.component(
    base_image="python:3.10.7-slim"
)
def compare_model(rf_metrics: dict, knn_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(rf_metrics)
    logging.info(knn_metrics)
    if rf_metrics.get("accuracy") > knn_metrics.get("accuracy"):
        return "RF"
    else :
        return "KNN"

### Upload Model and Metrics to Google Bucket 

In [20]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob(str(model.metadata["algo"]) + '_model' + str(model.metadata["file_type"])) 
    blob.upload_from_filename(model.path + str(model.metadata["file_type"]))       
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Define the Pipeline

In [21]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="disease-prdictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str):
    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_rf_job_run_op = train_rf(
        features=di_op.outputs["dataset"]
    )
    
     
    training_knn_job_run_op = train_knn(
        features=di_op.outputs["dataset"]
    )
    
    #pre_di_op = download_data(
    # project_id=project_id,
        #bucket=data_bucket,
        #file_name=testset_filename
    #).after(training_rf_job_run_op, training_knn_job_run_op)
        
        
    comp_model__op = compare_model(rf_metrics=training_rf_job_run_op.outputs["metrics"],
                                       knn_metrics=training_knn_job_run_op.outputs["metrics"]).after(training_rf_job_run_op, training_knn_job_run_op)  
    
    # defining the branching condition
    with dsl.If(comp_model__op.output=="RF"):
       # predict_rf_job_run_op = predict_model(
         #   model=training_rf_job_run_op.outputs["out_model"],      
         #   features=pre_di_op.outputs["dataset"]
        #)
        upload_model_rf_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_rf_job_run_op.outputs['out_model']
        ).after(comp_model__op)
        
    with dsl.If(comp_model__op.output=="KNN"):
       # predict_knn_job_run_op = predict_model(
       #     model=training_knn_job_run_op.outputs["out_model"],     
       #     features=pre_di_op.outputs["dataset"]
       # )
        upload_model_knn_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_knn_job_run_op.outputs['out_model']
        ).after(comp_model__op)  

#### Compile the pipeline into a JSON file

In [22]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='disease_predictor_training_pipeline.yaml')

#### Submit the pipeline run

In [23]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="disease_predictor_training_pipeline",
    enable_caching=False,
    template_path="disease_predictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2024',  # makesure to use your data bucket name 
        'trainset_filename': 'disease_diagnosis_final.csv',     # makesure to upload these to your data bucket from DE_TEAM16/
        #'testset_filename': 'prediction_set.csv',    # makesure to upload these to your data bucket from DE_TEAM16/
        'model_repo':'models_de2024' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/diabetes-prdictor-training-pipeline-v2-20241007202913?project=966204339179
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-20241007202913 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/966204339179/locations/us-central1/pipelineJobs/diabetes-prdictor-training-pipeline-v2-